<a href="https://colab.research.google.com/github/EffiSciencesResearch/ML4G-2.0/blob/master/workshops/hyperparameters/hyperparameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hyperparameters for a Binary classification problem

The goal of this workshop is to get familiar with:
- the concept of hyperparameters
- what the usual hyperparameters are for a neural network and what are their effects
- how to tune them and validate a choice of hyperparameters


## Hyperparameters

In the prerequisites you trained a very small model to learn the sinus function. In the process, you had to make many decisions, such as to use a degree 3 polynomial, to use a specific learning rate, etc.

The learning rate is an example of a **hyperparameter**, which will be described below. As a reminder, a regular parameter is an adjustable value with the special and extremely convenient property that we can differentiate the loss with respect to the parameter, allowing us to efficiently learn good values for the parameter using gradient descent. In other words, the process of training is a function that takes a dataset, a model architecture, and a random seed and outputs model parameters.

The learning rate, in contrast, cannot be determined by this scheme. As a hyperparameter, we need to introduce an outer loop that wraps the training loop to search for good learning rate values. This outer loop is called a hyperparameter search, and each iteration consists of testing different combinations of hyperparameters using a dataset of pairs of $(\text{hyperparameters}, \text{validation performance})$. Obtaining results for each iteration (a single pair) requires running the inner training loop.

Due to a fixed budget of ML researcher time and available compute, we are interested in a trade-off between the ML researcher time, the cost of running the search, and the cost of training the final model. Due to the vast search space and cost of obtaining data, we don't hope to find any sort of optimum but merely to improve upon our initial guesses enough to justify the cost.

In addition, a hyperparameter isn't necessarily a single continuous value like the learning rate. Discrete unordered choices such as padding type as well as discrete ordered choices such as the number of layers in the network or the width of each convolution are all common. You will also need to choose between functions for optimizers, nonlinearities, or learning rate scheduling, of which there are an infinite number of possibilities, requiring us to select a small subset to test.

More broadly, every design decision can be considered a hyperparameter, including how to preprocess the input data, the connectivity of different layers, the types of operations, etc. Papers such as [AmeobaNet](https://arxiv.org/pdf/1801.01548.pdf) demonstrated that it's possible to find architectures superior to human-designed ones.

In the second part of this workshop, you will be able to test various strategies for searching over hyperparameters.

## The dataset

We study first a binary classification problem, performed by a neural network. Each input has two real features, that is, they are points in 2D and the output can be only 0 or 1.

The training set contains 4000 examples, and the validation set, 1000.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

# Display figures on jupyter notebook
%matplotlib inline

In [ ]:
# We define a function to generate our synthetic the dataset, in the form of two interlaced spirals
# You don't need to understand this code, just run it


def spiral(phi):
    x = (phi + 1) * torch.cos(phi)
    y = phi * torch.sin(phi)
    return torch.cat((x, y), dim=1)


def generate_data(num_data):
    angles = torch.empty((num_data, 1)).uniform_(1, 15)
    data = spiral(angles)
    # add some noise to the data
    data += torch.empty((num_data, 2)).normal_(0.0, 0.4)
    labels = torch.zeros((num_data,), dtype=torch.int)
    # flip half of the points to create two classes
    data[num_data // 2 :, :] *= -1
    labels[num_data // 2 :] = 1
    return data, labels

In [ ]:
# Generate the training set with 4000 examples
x_train, y_train = generate_data(4000)

print("X_train", x_train.shape)
print("y_train", y_train.shape)

In [ ]:
def plot_data(x, y):
    """Plot labeled data points X and y. Label 1 is a red +, label 0 is a blue +."""
    plt.figure(figsize=(5, 5))
    plt.plot(x[y == 1, 0], x[y == 1, 1], "r+")
    plt.plot(x[y == 0, 0], x[y == 0, 1], "b+")

We can now invoke the `plot_data` function on the dataset previously generated to see what it looks like:

In [ ]:
plot_data(x_train, y_train)

We use the `TensorDataset` wrapper from pytorch, so that the framework can easily understand our tensors as a proper dataset.

A `Dataset` in PyTorch holds together the inputs and the corresponding labels (if any), and provides a way to access them.
It also interfaces neatly with the `DataLoader` class, which is used to load the data in batches, shuffle it, etc.

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

training_set = TensorDataset(x_train, y_train)

##  A neural network to classify the data

Here is a skeleton of a neural network with, by default, a single hidden layer. This is the model you'll try to improve during this exercise.

Look at the code and run it to see the structure, then follow the questions below to iteratively improve the model.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

At the first step, we define a neural network with just two layers. A useful tutorial for constructing models can be found [here](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#sphx-glr-beginner-blitz-neural-networks-tutorial-py).

In [ ]:
# Don't just run, but also read this code, it's code you would find again many times in the future

from typing_extensions import Literal


class Model(nn.Module):
    """
    A fully connected neural network with any number of layers.
    """

    NAME_TO_NONLINEARITY = {
        "relu": nn.ReLU,
        "sigmoid": nn.Sigmoid,
        "tanh": nn.Tanh,
    }

    def __init__(
        self, layers=[2, 10, 1], non_linearity: Literal["relu", "sigmoid", "tanh"] = "relu"
    ):
        super(Model, self).__init__()

        modules = []
        for input_dim, output_dim in zip(layers[:-1], layers[1:]):
            modules.append(nn.Linear(input_dim, output_dim))
            # After each linear layer, we apply a non-linearity
            modules.append(self.NAME_TO_NONLINEARITY[non_linearity]())

        # Remove the last non-linearity, since the last layer is the output layer
        self.layers = nn.Sequential(*modules[:-1])

    def forward(self, inputs):
        ouput = self.layers(inputs)

        # We want the model to predict 0 for one class and 1 for the other class
        # A Sigmoid activation function appropriate to map the output from [-inf, inf] to [0, 1]
        prediction = torch.sigmoid(ouput)
        return prediction

In [ ]:
# Create the model:
model = Model()

# Choose the hyperparameters for training:
num_epochs = 10
batch_size = 10

# Training criterion. This one is a mean squared error (MSE) loss between the output
# of the network and the target label
criterion = nn.MSELoss()

# Use SGD optimizer with a learning rate (lr) of 0.01
# It is initialized on our model
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

## Training the model
More information can be found [here](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#sphx-glr-beginner-blitz-neural-networks-tutorial-py) if needed.

In [ ]:
# tqdm is a library used to display progress bars. It's extremely useful when training.
from tqdm.notebook import tqdm


def train(
    num_epochs: int, batch_size: int, criterion, optimizer, model, dataset, verbose: bool = False
):
    """Train a model."""
    # Store the training errors
    train_losses = []
    # Create a DataLoader to iterate over the dataset in batches
    train_loader = DataLoader(dataset, batch_size, shuffle=True)

    for epoch in tqdm(range(num_epochs)):
        epoch_average_loss = 0
        # Each epoch, we iterate over the dataset once
        for x_batch, y_true in train_loader:
            # Compute the predictions.
            # Output shape is (batch_size, 1), so we squeeze the last dimension
            y_predicted = model(x_batch).squeeze(1)

            # The loss is how far the predictions are from the true labels
            loss = criterion(y_predicted, y_true.float())

            # Do gradient descent to minimize the loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Record the average loss for this batch
            epoch_average_loss += loss.item() * batch_size / len(dataset)

        train_losses.append(epoch_average_loss)

        if verbose:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_average_loss:.4f}")

    return train_losses

In [ ]:
train_losses = train(num_epochs, batch_size, criterion, optimizer, model, training_set, 1)

In [ ]:
# Plot the training error wrt. the number of epochs
plt.plot(range(1, num_epochs + 1), train_losses)
plt.xlabel("num_epochs")
plt.ylabel("Train error")
plt.title("Visualization of convergence")
plt.show()

## Evaluating the model on the validation set

We first evaluate the accuracy on a validation set, to see how the model performs on unseen data.

In [ ]:
# Generate 1000 validation datapoints
x_val, y_val = generate_data(1000)


def get_accuracy(model, x=x_val, y=y_val):
    """Compute the accuracy of the model on a dataset."""
    # Compute the predictions, without keeping track of the gradients
    with torch.no_grad():
        y_predicted = model(x).squeeze(1)

    # The predictions are in [0, 1] and the labels are either 0 or 1
    # So we round the predictions to get the predicted labels
    y_predicted = torch.round(y_predicted)

    # Compute the accuracy by counting the number of correct predictions
    accuracy = (y_predicted == y).sum().item() / len(y)

    print(f"Accuracy on {len(y)} examples: {accuracy:.2%}")
    return accuracy

In [ ]:
get_accuracy(model);

Then we visualize what the model has learned by plotting all the predictions.

In [ ]:
def compare_predictions(model, x=x_val, y_real=y_val):
    """Compare the prediction with real labels."""

    with torch.no_grad():
        y_predicted = model(x).squeeze(1)

    plt.figure(figsize=(10, 5))

    reds = y_real > 0.5
    plt.subplot(121)
    plt.plot(x[reds, 0], x[reds, 1], "r+")
    plt.plot(x[~reds, 0], x[~reds, 1], "b+")
    plt.title("real data")

    reds = y_predicted > 0.5
    plt.subplot(122)
    plt.plot(x[reds, 0], x[reds, 1], "r+")
    plt.plot(x[~reds, 0], x[~reds, 1], "b+")
    plt.title("predicted data")

    plt.show()

In [ ]:
compare_predictions(model)

## Meta-optimisation

We defined a lot a hyper-parameters (learning rate, layer sizes...) in the previous section. We will now try to find the best combination of hyper-parameters.

> WARNING: For this exercise to be maximally useful, before you start answering the questions, try to make predictions about the impact of each meta-parameter.
Afterwards, you can check that the predictions were correct.

Bonus: if you want, you can make your predictions on [FateBook](https://fatebook.io).


Questions:
- Meta optimization: The goal of this tutorial is to get a summary table of the impact of each meta-parameter by clicking once on the "Run All" button.
To do this, you need to think about the difference between the grid search strategy and the sensitivity analysis strategy? Which strategy is more suitable in case there are a lot of meta parameters? Try to implement this strategy in the following.
- Why do you need the test dataset in addition to the validation dataset?

### Exercise 1: Impact of the optimizer

Retrain the model by using different hyperparameters, you can change them in the previous sections definition, or put everything you need in the cell below for convenience.

Try to see the impact of the following factors:
* Use different batch size from 10 to 400
* Try different values of the learning rate (between 0.001 and 10), and see how these impact the training process.
* Change the duration of the training by increasing the number of epochs
* Try other optimizers, such as [Adam](https://pytorch.org/docs/stable/optim.html?highlight=adam#torch.optim.Adam) or [RMSprop](https://pytorch.org/docs/stable/optim.html?highlight=rmsprop#torch.optim.RMSprop)

### Exercise 2: Impact of the architecture of the model

The class `Model` is the definition of your model.
Retrain the model by using different architectures, similarly as before.

Try out different architectures and
see the impact of the following factors:

* Try to add more layers (1, 2, 3, more ?)
* Try to different activation functions ([sigmoid](https://pytorch.org/docs/stable/nn.functional.html#torch.nn.functional.sigmoid), [tanh](https://pytorch.org/docs/stable/nn.functional.html#torch.nn.functional.tanh), [relu](https://pytorch.org/docs/stable/nn.functional.html#torch.nn.functional.relu), etc.)
* Try to change the number of neurons for each layer (5, 10, 20, more ?)
* Do all network architectures react the same way to different learning rates?


**Note:** These changes may interact with your previous choices of hyperparameters, and you may need to change them as well!

### Exercise 3: Impact of the loss function

The current model uses a mean square error (MSE) loss. While this loss can be used in this case, it is now rarely used for classification, and instead a Binary Cross Entropy (BCE) is used. It consists in interpreting the output of the network as the probability $p(y | x)$ of the point $x$ to belong to the class $y$, and in maximizing the probability to be correct for all samples $x$, that is, in maximizing $\displaystyle \prod_{(x,y) \in Dataset} p(y|x)$. Applying $-\log$ to this quantity, we obtain the following criterion to minimize:

$$ \sum_{(x,y) \in Dataset} - \log p(y | x) $$

This is implemented as such by the [BCELoss](https://pytorch.org/docs/stable/nn.html?highlight=bce#torch.nn.BCELoss) of pytorch. Note that this criterion requires its input to be a probability, i.e. in $[0,1]$, which requires the use of an appropriate activation function beforehand, e.g., a sigmoid.

It turns out that, for numerical stability reasons, it is better to incorporate this sigmoid and the BCELoss into a single function; this is done by the [BCEWithLogitsLoss](https://pytorch.org/docs/stable/nn.html?highlight=bcewithlogit#torch.nn.BCEWithLogitsLoss). Try to replace the MSE by this one and see how this changes the behavior in the network. This can also interact with the changes of the two previous exercices.

**Note:** As a consequence, when using the BCEWithLogitsLoss, the last layer of your network should not be followed by an activation function, as BCEWithLogitsLoss already adds a sigmoid.

### Exercise 4: Prediction on test set

Once you have a model that seems satisfying on the validation dataset, you SHOULD evaluate it on a test dataset that has never been used before, to obtain a final accuracy value.